In [ ]:
# =============================================================================
# Messages - The Building Blocks of LLM Conversations (LangChain 1.0+)
# =============================================================================
"""
=====================================================================
Understanding LangChain Messages
=====================================================================

Messages are the fundamental units of communication with LLMs.
Every conversation is represented as a list of messages.

Message Types:
--------------
1. SystemMessage - Sets the AI's behavior/persona (invisible to users)
2. HumanMessage - User's input/questions
3. AIMessage - Model's responses
4. ToolMessage - Results from tool execution
5. FunctionMessage - (Deprecated) Old way of returning function results

Message Flow in a Conversation:
-------------------------------
    [SystemMessage]     "You are a helpful assistant"
           │
           ▼
    [HumanMessage]      "What's 2+2?"
           │
           ▼
    [AIMessage]         "2+2 equals 4"
           │
           ▼
    [HumanMessage]      "And 3+3?"
           │
           ▼
    [AIMessage]         "3+3 equals 6"

Why Messages Matter:
--------------------
- LLMs are stateless - they don't remember previous turns
- YOU must pass the full conversation history each time
- Message order matters (system → human → ai → human → ai...)
- Context window limits how many messages you can include

Updated for LangChain 1.0+ (2025-2026)
"""

import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

print("✅ Environment configured for messages examples")

In [ ]:
# =============================================================================
# Core Message Types (LangChain 1.0+)
# =============================================================================
"""
The Four Essential Message Types
--------------------------------
Import from langchain_core.messages (NOT langchain.messages)

LangChain 1.0+ uses langchain_core for all core abstractions.
"""

from langchain_core.messages import (
    SystemMessage,
    HumanMessage,
    AIMessage,
    ToolMessage,
)
from langchain.chat_models import init_chat_model

# Initialize model
model = init_chat_model("gpt-4o-mini", model_provider="openai", temperature=0.7)

# =============================================================================
# 1. SystemMessage - Sets AI Behavior
# =============================================================================
"""
SystemMessage:
- First message in the conversation (usually)
- Defines the AI's persona, rules, and behavior
- Not visible to the user in most UIs
- Affects ALL subsequent responses
"""

system_msg = SystemMessage(
    content="""You are a helpful assistant who specializes in French culture.
    Always include a fun fact about France when relevant.
    Keep responses concise but informative."""
)

print("=" * 60)
print("1. SystemMessage Example")
print("=" * 60)
print(f"Type: {type(system_msg).__name__}")
print(f"Content: {system_msg.content[:100]}...")

# =============================================================================
# 2. HumanMessage - User Input
# =============================================================================
"""
HumanMessage:
- Represents what the user says/asks
- Can include optional 'name' for multi-user scenarios
- The main input that drives the conversation
"""

human_msg = HumanMessage(
    content="What is the capital of France?",
    name="user_naveen"  # Optional: useful for multi-user chats
)

print("\n" + "=" * 60)
print("2. HumanMessage Example")
print("=" * 60)
print(f"Type: {type(human_msg).__name__}")
print(f"Content: {human_msg.content}")
print(f"Name: {human_msg.name}")

# =============================================================================
# 3. AIMessage - Model Response
# =============================================================================
"""
AIMessage:
- Represents the model's response
- Can contain tool_calls if the model wants to use tools
- Used to build conversation history
"""

# Manual AIMessage (for building history)
ai_msg = AIMessage(
    content="The capital of France is Paris! Fun fact: Paris is called 'La Ville Lumière' (The City of Light)."
)

print("\n" + "=" * 60)
print("3. AIMessage Example")
print("=" * 60)
print(f"Type: {type(ai_msg).__name__}")
print(f"Content: {ai_msg.content}")
print(f"Tool calls: {ai_msg.tool_calls}")

# =============================================================================
# Complete Conversation Example
# =============================================================================

messages = [
    system_msg,
    human_msg,
]

print("\n" + "=" * 60)
print("Complete Conversation")
print("=" * 60)

response = model.invoke(messages)
print(f"User: {human_msg.content}")
print(f"AI: {response.content}")

In [ ]:
# =============================================================================
# 4. ToolMessage - Tool Execution Results
# =============================================================================
"""
ToolMessage:
- Contains results from tool execution
- MUST include tool_call_id matching the original call
- Sent back to the model so it can use the result

Tool Message Flow:
------------------
[HumanMessage] → [AIMessage with tool_calls] → [ToolMessage with result] → [AIMessage final]
"""

# Simulating a tool call scenario
from langchain_core.tools import tool

@tool
def get_population(city: str) -> str:
    """Get the population of a city."""
    populations = {"Paris": "2.1 million", "London": "8.8 million", "Tokyo": "13.9 million"}
    return populations.get(city, "Unknown")

# Bind tool to model
model_with_tools = model.bind_tools([get_population])

# Simulate the complete flow
messages_with_tools = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="What's the population of Paris?")
]

print("=" * 60)
print("4. ToolMessage Example - Complete Flow")
print("=" * 60)

# Step 1: Get model response (may contain tool calls)
response = model_with_tools.invoke(messages_with_tools)
messages_with_tools.append(response)

print(f"\n📝 User: What's the population of Paris?")

if response.tool_calls:
    print(f"\n🔧 Model wants to use tool:")
    for tc in response.tool_calls:
        print(f"   Tool: {tc['name']}")
        print(f"   Args: {tc['args']}")
        print(f"   Call ID: {tc['id']}")
        
        # Step 2: Execute tool and create ToolMessage
        result = get_population.invoke(tc['args'])
        tool_msg = ToolMessage(
            content=str(result),
            tool_call_id=tc['id']  # MUST match!
        )
        messages_with_tools.append(tool_msg)
        
        print(f"\n📊 Tool Result:")
        print(f"   Content: {tool_msg.content}")
        print(f"   Tool Call ID: {tool_msg.tool_call_id}")
    
    # Step 3: Get final response
    final_response = model_with_tools.invoke(messages_with_tools)
    print(f"\n✅ Final Answer: {final_response.content}")
else:
    print(f"\n✅ Direct Answer: {response.content}")

In [ ]:
# =============================================================================
# Building Multi-Turn Conversations
# =============================================================================
"""
Conversation Memory Pattern
---------------------------
Since LLMs are stateless, YOU manage the conversation history.
Each call must include all relevant previous messages.

Key Concepts:
- Append new messages to the list after each turn
- Include both user messages AND AI responses
- System message usually stays constant
"""

def chat_with_memory():
    """Simple chatbot with conversation memory."""
    
    conversation = [
        SystemMessage(content="You are a helpful math tutor. Be encouraging and explain step by step.")
    ]
    
    # Simulated conversation
    user_inputs = [
        "What is 15% of 80?",
        "Can you explain how you calculated that?",
        "Now what's 20% of the same number?"
    ]
    
    print("=" * 60)
    print("Multi-Turn Conversation Example")
    print("=" * 60)
    
    for user_input in user_inputs:
        # Add user message
        conversation.append(HumanMessage(content=user_input))
        
        # Get AI response
        response = model.invoke(conversation)
        
        # Add AI response to history
        conversation.append(response)
        
        print(f"\n👤 User: {user_input}")
        print(f"🤖 AI: {response.content[:200]}..." if len(response.content) > 200 else f"🤖 AI: {response.content}")
    
    print(f"\n📊 Total messages in conversation: {len(conversation)}")
    return conversation

# Run the conversation
conversation_history = chat_with_memory()

In [ ]:
# =============================================================================
# Message Placeholders and Prompt Templates
# =============================================================================
"""
MessagesPlaceholder - Dynamic Message Insertion
-----------------------------------------------
Used in ChatPromptTemplate to insert variable-length message lists.
Perfect for conversation history in chatbots.

Pattern:
ChatPromptTemplate.from_messages([
    ("system", "..."),
    MessagesPlaceholder("history"),  # Insert conversation here
    ("human", "{input}")
])
"""

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Template with placeholder for history
chat_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Be concise."),
    MessagesPlaceholder("chat_history"),  # Dynamic history goes here
    ("human", "{user_input}")
])

print("=" * 60)
print("MessagesPlaceholder Example")
print("=" * 60)

# Build conversation history
history = [
    HumanMessage(content="My name is Naveen"),
    AIMessage(content="Nice to meet you, Naveen!"),
    HumanMessage(content="I'm learning LangChain"),
    AIMessage(content="That's great! LangChain is a powerful framework for building LLM applications."),
]

# Format with history and new input
formatted = chat_template.invoke({
    "chat_history": history,
    "user_input": "What's my name and what am I learning?"
})

print(f"\n📋 Formatted messages:")
for i, msg in enumerate(formatted.messages):
    msg_type = type(msg).__name__
    content = msg.content[:80] + "..." if len(msg.content) > 80 else msg.content
    print(f"   {i+1}. [{msg_type}] {content}")

# Get response
response = model.invoke(formatted)
print(f"\n✅ Response: {response.content}")

In [ ]:
# =============================================================================
# Message Trimming for Long Conversations
# =============================================================================
"""
Managing Context Window Limits
------------------------------
LLMs have token limits. Long conversations must be trimmed.

Strategies:
1. Keep last N messages (simple)
2. Keep system + last N (preserve persona)
3. Summarize old messages (advanced)
4. Use trim_messages() utility
"""

from langchain_core.messages import trim_messages

# Simulate a long conversation
long_conversation = [
    SystemMessage(content="You are a helpful assistant."),
]

# Add many messages
for i in range(10):
    long_conversation.append(HumanMessage(content=f"Question {i+1}: What is {i+1} + {i+1}?"))
    long_conversation.append(AIMessage(content=f"Answer: {i+1} + {i+1} = {(i+1)*2}"))

print("=" * 60)
print("Message Trimming Example")
print("=" * 60)
print(f"\n📊 Original conversation: {len(long_conversation)} messages")

# Method 1: Simple slicing - keep system + last 6 messages
trimmed_simple = [long_conversation[0]] + long_conversation[-6:]
print(f"📊 After simple trim: {len(trimmed_simple)} messages")

# Method 2: Using trim_messages utility (LangChain built-in)
# This is more sophisticated - it can trim based on token count
trimmed_utility = trim_messages(
    long_conversation,
    max_tokens=500,  # Approximate token limit
    strategy="last",  # Keep last messages
    token_counter=len,  # Simple character count (use tiktoken for accuracy)
    include_system=True,  # Always keep system message
    allow_partial=False,  # Don't cut messages in half
)

print(f"📊 After trim_messages: {len(trimmed_utility)} messages")

# Show trimmed conversation
print("\n📋 Trimmed conversation:")
for i, msg in enumerate(trimmed_simple):
    msg_type = type(msg).__name__[:6]
    content = msg.content[:50] + "..." if len(msg.content) > 50 else msg.content
    print(f"   {i+1}. [{msg_type}] {content}")

In [ ]:
# =============================================================================
# Summary: Messages in LangChain 1.0+
# =============================================================================
"""
=====================================================================
KEY TAKEAWAYS - Messages in LangChain
=====================================================================

1. MESSAGE TYPES:
   ---------------
   from langchain_core.messages import (
       SystemMessage,   # AI persona/rules
       HumanMessage,    # User input
       AIMessage,       # Model response
       ToolMessage,     # Tool execution result
   )

2. BASIC CONVERSATION:
   --------------------
   messages = [
       SystemMessage(content="You are helpful."),
       HumanMessage(content="Hello!"),
   ]
   response = model.invoke(messages)

3. MULTI-TURN PATTERN:
   --------------------
   conversation = [SystemMessage(...)]
   
   while True:
       user_input = get_input()
       conversation.append(HumanMessage(content=user_input))
       response = model.invoke(conversation)
       conversation.append(response)

4. TOOL MESSAGE FLOW:
   -------------------
   [Human] → [AI with tool_calls] → [Tool result] → [AI final answer]
   
   ToolMessage(
       content="result",
       tool_call_id="call_abc123"  # MUST match AIMessage.tool_calls[].id
   )

5. MESSAGES PLACEHOLDER:
   ----------------------
   from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
   
   template = ChatPromptTemplate.from_messages([
       ("system", "..."),
       MessagesPlaceholder("history"),
       ("human", "{input}")
   ])

6. TRIMMING FOR CONTEXT LIMITS:
   -----------------------------
   from langchain_core.messages import trim_messages
   
   trimmed = trim_messages(
       messages,
       max_tokens=4000,
       strategy="last",
       include_system=True
   )

Common Imports:
---------------
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import trim_messages

=====================================================================
"""

print("=" * 60)
print("Messages Module Complete!")
print("=" * 60)
print("""
Next Steps:
-----------
1. 5-structureoutput.ipynb - Getting structured data from LLMs
2. 6-middleware.ipynb - Request/response middleware

Key Points:
-----------
- Messages are the ONLY way to communicate with LLMs
- YOU manage conversation history (LLMs are stateless)
- Always match tool_call_id in ToolMessages
- Trim messages when approaching context limits
""")